<a href="https://colab.research.google.com/github/jvonrad/Machine-Vision-PushUp-Detector/blob/main/Machine_Vision_Final_Lab_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3 -q
!pip install opencv-python torch numpy torchvision
!pip install opencv-python
!pip install 'mediapipe==0.10.13'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 148.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, b

## Download the data

The data for this assignment has been made available and is downloadable to disk by running the below cell.

In [2]:
import re
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
import cv2


def download_videos():
    # Connect to S3 without authentication (public bucket)
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    bucket_name = 'prism-mvta'
    prefix = 'training-and-validation-data/'
    download_dir = './video-data'

    os.makedirs(download_dir, exist_ok=True)

    # List all objects in the S3 path
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    video_names = []

    for page in pages:
        if 'Contents' not in page:
            print("No files found at the specified path! Go and complain to the TAs!")
            break

        for obj in page['Contents']:
            key = obj['Key']
            filename = os.path.basename(key)

            if not filename:
                continue

            video_names.append(filename)

            local_path = os.path.join(download_dir, filename)
            print(f"Downloading: {filename}")
            s3.download_file(bucket_name, key, local_path)

    print("\n" + "="*50)
    print("Downloaded videos:")
    print("="*50)
    for name in video_names:
        print(name)

    print(f"\nTotal: {len(video_names)} files")

download_videos()


Downloading: 1_dksksjfwijf.mp4
Downloading: 2_dfsaeklnvvalkej.mp4
Downloading: 2_difficult_2.mp4
Downloading: 2_difficult_sdafkljsalkfj.mp4
Downloading: 2_dkdjwkndkfw.mp4
Downloading: 2_dkdmkejkeimdh.mp4
Downloading: 2_dkjd823kjf.mp4
Downloading: 2_dsalkfjalwkenlke.mp4
Downloading: 2_kling_20251205_Text_to_Video_On_a_sandy_4976_0.mp4
Downloading: 2_kling_20251206_Text_to_Video_Generate_a_71_1.mp4
Downloading: 2_sadfasjldkfjaseifj.mp4
Downloading: 2_sdafkjaslkclaksdjkas.mp4
Downloading: 2_sdfkjsaleijflaskdjf.mp4
Downloading: 2_sdjfhafsldkjhjk.mp4
Downloading: 2_sdkjdsflkjfwa.mp4
Downloading: 2_sdlfjlewlkjkj.mp4
Downloading: 2_sdlkjsaelijfksdjf.mp4
Downloading: 3_asldkfjalwieaskdfaskdf.mp4
Downloading: 3_dkk873lkjlksajdf.mp4
Downloading: 3_dsjlaeijlksjdfie.mp4
Downloading: 3_dsksdfjbvsdkj.mp4
Downloading: 3_dslkaldskjflakjs.mp4
Downloading: 3_ewdfkjwaeoihjlkasdjf.mp4
Downloading: 3_kling_20251205_Text_to_Video_In_a_grass_4697_0.mp4
Downloading: 3_kling_20251205_Text_to_Video_On_a_playg_5

## MediaPipe Pose Model

In [3]:
import mediapipe as mp

class MediaPipePoseExtractor:
    def __init__(self, static_image_mode=False, model_complexity=1, smooth_landmarks=True):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=static_image_mode,
            model_complexity=model_complexity,
            smooth_landmarks=smooth_landmarks,
            enable_segmentation=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

    def extract(self, video_path, max_frames=None, stride=1, include_visibility=True):
        cap = cv2.VideoCapture(video_path)
        keypoints = []
        frame_idx = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % stride != 0:
                frame_idx += 1
                continue

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.pose.process(rgb)

            if results.pose_landmarks is None:
                # if detection fails, fill zeros
                if include_visibility:
                    kp = np.zeros((33, 3), dtype=np.float32)
                else:
                    kp = np.zeros((33, 2), dtype=np.float32)
            else:
                lm = results.pose_landmarks.landmark
                if include_visibility:
                    kp = np.array([[p.x, p.y, p.visibility]
                                  for p in lm], dtype=np.float32)
                else:
                    kp = np.array([[p.x, p.y] for p in lm], dtype=np.float32)

            keypoints.append(kp)

            frame_idx += 1
            if max_frames is not None and len(keypoints) >= max_frames:
                break

        cap.release()
        return np.stack(keypoints, axis=0)  # (T, 33, D)

These videos are now available in the folder "video-data". You can click on the folder icon on the left-hand-side of this screen to see the videos in a file explorer.

# Create your Datasets and Dataloaders

Some example code for approaching the first *two* TODOs is given below just to get you started. No starter code is given for the third TODO.

Note, the below code is very rough skeleton code. Make no assumptions as to the correct manner to architect your model based on the structure of this code.

Please feel free to (if not encouraged to) change every single line of the below code (change it to best suit your chosen model architecture, in the next section).

### TODO 1 (This is mostly already done for you - Please see the v1 provided below)

Each video in the folder is prefixed by a number. That number corresponds to the number of distinct pushups visible in the video. Write code to iterate over each video in the folder, and extract the corresponding target associated with the video.

### TODO 2 (This is also mostly already done for you - Please see the v1 provided below)


Divide the data into training and validation sets.

Optionally, you can also create out your own test set to assess your performance.

### TODO 3

Any preprocessing or augmentation of your data which you deem required, should (probably) go here. You are also free to include your data-augmentation code later, though doing it before creating your dataloaders is probably a good idea.

If you complete this TODO, to maintain experimental hygiene, feel free to modify the code which was provided for TODOs 1 and 2.

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import os
import cv2
import numpy as np


class VideoDataset(Dataset):
    """Dataset for loading videos from a folder. Labels from filename prefix."""

    def __init__(self, video_dir, frame_size=(224, 224), transform=None):
        self.video_dir = video_dir
        self.frame_size = frame_size
        self.transform = transform

        self.video_files = [
            f for f in os.listdir(video_dir)
            if f.endswith(('.mp4', '.avi', '.mov'))
        ]

        self.labels = [
            int(f.split('_')[0]) for f in self.video_files
        ]

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path = os.path.join(self.video_dir, self.video_files[idx])
        frames = self._load_video(video_path)
        label = self.labels[idx]

        if self.transform:
            frames = self.transform(frames)

        return frames, label

    def _load_video(self, path):
        cap = cv2.VideoCapture(path)

        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (self.frame_size[1], self.frame_size[0]))
            frames.append(frame)

        cap.release()

        frames = torch.from_numpy(np.array(frames)).permute(3, 0, 1, 2).float() / 255.0

        return frames


import os
import cv2
import numpy as np
from pathlib import Path
import random

def augment_and_save_video(
    in_path: str,
    out_path: str,
    crop_scale_range=(0.80, 1.00),
    flip_prob=0.5,
    rot_range=(-10, 10),        # degrees
    scale_range=(0.9, 1.1),     # scaling factor
    trans_frac=0.1,          # fraction of width/height
    seed=None
):
    if seed is not None:
        random.seed(seed)

    cap = cv2.VideoCapture(in_path)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {in_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # force output extension to .avi
    if out_path.lower().endswith(".mp4"):
        out_path = out_path[:-4] + ".avi"

    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    out = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    if not out.isOpened():
        cap.release()
        raise RuntimeError(f"Could not open VideoWriter: {out_path}")

    # ---------- Sample augmentation parameters ONCE per video ----------
    do_flip = (random.random() < flip_prob)

    # cropping
    scale_crop = random.uniform(*crop_scale_range)
    crop_w = int(w * scale_crop)
    crop_h = int(h * scale_crop)
    x0 = 0 if crop_w == w else random.randint(0, w - crop_w)
    y0 = 0 if crop_h == h else random.randint(0, h - crop_h)

    # camera geometry jitter
    rot_deg = random.uniform(*rot_range)
    scale   = random.uniform(*scale_range)
    tx      = random.uniform(-trans_frac, trans_frac) * w
    ty      = random.uniform(-trans_frac, trans_frac) * h

    M = cv2.getRotationMatrix2D((w / 2, h / 2), rot_deg, scale)
    M[:, 2] += (tx, ty)

    # --------------------------------------------------------------

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 1) camera geometry warp (simulate camera movement)
        frame = cv2.warpAffine(
            frame, M, (w, h),
            flags=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_REFLECT101
        )

        # 2) crop
        cropped = frame[y0:y0 + crop_h, x0:x0 + crop_w]

        # 3) resize back to original size
        resized = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)

        # 4) horizontal flip
        if do_flip:
            resized = cv2.flip(resized, 1)

        out.write(resized)

    cap.release()
    out.release()


def build_augmented_dataset(
    src_dir="./video-data",
    dst_dir="./video-data-aug",
    copies_per_video=2,
    crop_scale_range=(0.80, 1.00),
    flip_prob=0.5,
    seed=42
):
    src_dir = Path(src_dir)
    dst_dir = Path(dst_dir)
    dst_dir.mkdir(parents=True, exist_ok=True)

    rng = random.Random(seed)

    video_files = sorted([p for p in src_dir.iterdir() if p.suffix.lower() == ".mp4"])
    print(f"Found {len(video_files)} original videos")

    # 1) copy originals into dst_dir (so dst_dir becomes "expanded set")
    for vid in video_files:
        out_orig = dst_dir / vid.name
        if not out_orig.exists():
            out_orig.write_bytes(vid.read_bytes())

    # 2) create augmented copies
    for vid in video_files:
        stem = vid.stem  # e.g. "4_pushup_...."
        for k in range(copies_per_video):
            out_name = f"{stem}__aug{k}.mp4"
            out_path = dst_dir / out_name

            if out_path.exists():
                continue

            # different seed per output
            aug_seed = rng.randint(0, 10**9)
            augment_and_save_video(
                str(vid),
                str(out_path),
                crop_scale_range=crop_scale_range,
                flip_prob=flip_prob,
                seed=aug_seed
            )

        print(f"Augmented: {vid.name}")

    print(f"Expanded dataset written to: {dst_dir} (originals + aug copies)")
    return str(dst_dir)

import os
import random
from pathlib import Path
from collections import defaultdict

def parse_label_from_filename(fname: str) -> int:
    # "4_pushup_abc.mp4" -> 4
    return int(fname.split("_")[0])

def build_augmented_dataset_balanced(
    src_dir="./video-data",
    dst_dir="./video-data-aug",
    target_per_class=50,
    crop_scale_range=(0.80, 1.00),
    flip_prob=0.5,
    rot_range=(-10, 10),
    scale_range=(0.9, 1.1),
    trans_frac=0.1,
    seed=42,
    copy_originals=True,
):
    src_dir = Path(src_dir)
    dst_dir = Path(dst_dir)
    dst_dir.mkdir(parents=True, exist_ok=True)

    rng = random.Random(seed)

    # --- gather originals by class ---
    originals_by_label = defaultdict(list)
    for p in sorted(src_dir.iterdir()):
        if p.suffix.lower() not in [".mp4", ".avi"]:
            continue
        try:
            y = parse_label_from_filename(p.name)
        except Exception:
            continue
        originals_by_label[y].append(p)

    if not originals_by_label:
        raise RuntimeError(f"No .mp4 videos found in {src_dir}")

    print("Original counts per class:")
    for y in sorted(originals_by_label):
        print(f"  {y}: {len(originals_by_label[y])}")

    # --- optionally copy originals into dst ---
    if copy_originals:
        for y, vids in originals_by_label.items():
            for vid in vids:
                out_orig = dst_dir / vid.name
                if not out_orig.exists():
                    out_orig.write_bytes(vid.read_bytes())

    # --- find how many already exist in dst_dir (originals + prior augs) ---
    existing_by_label = defaultdict(int)
    for p in dst_dir.iterdir():
        if p.suffix.lower() not in [".mp4", ".avi"]:
            continue
        try:
            y = parse_label_from_filename(p.name)
        except Exception:
            continue
        existing_by_label[y] += 1

    print("\nExisting counts in dst_dir before new augs:")
    for y in sorted(originals_by_label):
        print(f"  {y}: {existing_by_label[y]}")

    # --- generate more until each class hits target_per_class ---
    for y in sorted(originals_by_label):
        have = existing_by_label[y]
        need = max(0, target_per_class - have)

        if need == 0:
            print(f"\nClass {y}: already has {have} >= {target_per_class}, skipping.")
            continue

        vids = originals_by_label[y]
        if len(vids) == 0:
            print(f"\nClass {y}: no originals, cannot augment.")
            continue

        print(f"\nClass {y}: need {need} more to reach {target_per_class}.")

        # round-robin across originals so you don't over-augment just one clip
        for i in range(need):
            src_vid = vids[i % len(vids)]
            stem = src_vid.stem  # e.g. "4_pushup_abc"
            aug_id = rng.randint(0, 10**9)

            out_name = f"{stem}__balaug_{aug_id}.avi"
            out_path = dst_dir / out_name

            # avoid extremely rare collision
            while out_path.exists():
                aug_id = rng.randint(0, 10**9)
                out_name = f"{stem}__balaug_{aug_id}.avi"
                out_path = dst_dir / out_name

            augment_and_save_video(
                in_path=str(src_vid),
                out_path=str(out_path),
                crop_scale_range=crop_scale_range,
                flip_prob=flip_prob,
                rot_range=rot_range,
                scale_range=scale_range,
                trans_frac=trans_frac,
                seed=rng.randint(0, 10**9),
            )

        print(f"  -> class {y} now should be ~{target_per_class} in {dst_dir}")

    print(f"\n✅ Balanced augmented dataset written to: {dst_dir}")
    return str(dst_dir)


## Cached Pose Data

In [5]:
from torch.utils.data import WeightedRandomSampler


class CachedPoseDataset(Dataset):
    """
    Pose-only dataset.
    Expects files like: 4_something__aug1.npy
    Label = int(prefix before first underscore)
    """

    def __init__(self, pose_dir):
        self.pose_dir = Path(pose_dir)
        self.pose_files = sorted(self.pose_dir.glob("*.npy"))

        if len(self.pose_files) == 0:
            raise RuntimeError(f"No .npy files found in {pose_dir}")

        self.labels = [int(p.name.split("_")[0]) for p in self.pose_files]

    def __len__(self):
        return len(self.pose_files)

    def __getitem__(self, idx):
        p = self.pose_files[idx]
        kp = np.load(p).astype(np.float32)          # (T,33,3)
        kp = torch.from_numpy(kp).float()
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return kp, y


def resample_pose_to_length(pose: np.ndarray, target_len: int = 300) -> np.ndarray:
    """
    pose: (T, 33, 3)
    returns: (target_len, 33, 3)
    """
    T, V, D = pose.shape
    if T == target_len:
        return pose.astype(np.float32)

    # Old and new time indices
    old_x = np.linspace(0, 1, T)
    new_x = np.linspace(0, 1, target_len)

    out = np.zeros((target_len, V, D), dtype=np.float32)
    for v in range(V):
        for d in range(D):
            out[:, v, d] = np.interp(new_x, old_x, pose[:, v, d])

    return out


def pose_collate_fn(batch):
    kps, labels = zip(*batch)
    return torch.stack(kps, dim=0), torch.stack(labels, dim=0)


def get_dataloaders(pose_dir, batch_size=4, val_split=0.01, frame_size=(224, 224)):
    """Create train and validation dataloaders."""

    full_dataset = CachedPoseDataset(pose_dir)
    val_size = int(len(full_dataset) * val_split)
    train_size = len(full_dataset) - val_size

    train_dataset, val_dataset = random_split(
        full_dataset,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        collate_fn=pose_collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        collate_fn=pose_collate_fn
    )

    print(
        f"Train: {len(train_dataset)} videos, Val: {len(val_dataset)} videos\n")

    return train_loader, val_loader


video_dir = './video-data'

# Create a Model

For this assignment, we request you use PyTorch. Below is an example of how to instantiate a very basic PyTorch model.

Note, this model below needs a _lot_ of work.

Please include your code for creating your model below.

The only constraint here is that you define a Python object which inherits from a PyTorch nn.Module object. Beyond that, please feel free to implement anything you like: Transformer, Vision Transformer, MLP, CNN, etc.

### TODO 4

Create your model.

In [6]:

import torch
import torch.nn as nn
from huggingface_hub import HfApi, hf_hub_download


####################
# Video Classifier
####################

# This one was initially used during development
class PoseTemporalCNN(nn.Module):
    def __init__(self, num_classes=10, num_joints=33, dim=3):
        super().__init__()
        in_feats = num_joints * dim

        self.net = nn.Sequential(
            nn.Conv1d(in_feats, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
        )
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        # x: (B, T, 33, 3)
        B, T, V, D = x.shape
        x = x.view(B, T, V * D).transpose(1, 2)  # (B, V*D, T)
        x = self.net(x).squeeze(-1)              # (B, 256)
        return self.fc(x)

# This is the one we actually use
class PoseGRUClassifier(nn.Module):
    def __init__(self, num_classes=11, input_dim=99, hidden=256):
        super().__init__()
        self.gru = nn.GRU(input_size=input_dim, hidden_size=hidden,
                          batch_first=True, bidirectional=True)
        self.head = nn.Sequential(
            nn.Linear(hidden * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        # x: (B, T, 33, 3)
        B, T, V, D = x.shape
        x = x.view(B, T, V * D)  # (B, T, 99)

        out, _ = self.gru(x)
        vid = out.mean(dim=1)
        return self.head(vid)

## Cache Pose Data

In [7]:
from pathlib import Path


def cache_all_poses(video_dir="./video-data", pose_dir="./pose-data",
                    target_frames=300, stride=2, include_visibility=True):

    video_dir = Path(video_dir)
    pose_dir = Path(pose_dir)
    pose_dir.mkdir(exist_ok=True)

    extractor = MediaPipePoseExtractor(
        model_complexity=1, smooth_landmarks=True)

    for vid in video_dir.iterdir():
        if vid.suffix.lower() not in [".mp4", ".avi", ".mov"]:
            continue

        out = pose_dir / (vid.stem + ".npy")
        if out.exists():
            continue

        print("Caching", vid.name)

        kp = extractor.extract(str(vid), max_frames=target_frames,
                               stride=stride, include_visibility=include_visibility)

        kp = resample_pose_to_length(kp, target_len=target_frames)

        np.save(out, kp.astype(np.float32))


def base_id_from_filename(fname: str) -> str:
    # "4_pushup_001__aug0.mp4" -> "4_pushup_001"
    stem = os.path.splitext(fname)[0]
    stem = re.sub(r"__aug\d+$", "", stem)
    return stem

## Pose Stitching for Videos of 8,9 and 10

In [8]:

from pathlib import Path
import numpy as np
import shutil


def self_stitch_pose(pose: np.ndarray, times: int, fade: int = 15) -> np.ndarray:
    """
    Repeat the same pose sequence `times` and cross-fade at boundaries.
    pose: (T, 33, 3)
    """
    assert times >= 1
    pose = pose.astype(np.float32)

    out = pose
    for _ in range(times - 1):
        nxt = pose

        f = min(fade, len(out), len(nxt))
        if f <= 1:
            out = np.concatenate([out, nxt], axis=0)
            continue

        a = out[:-f]
        b1 = out[-f:]
        b2 = nxt[:f]
        c = nxt[f:]

        w = np.linspace(0, 1, f, dtype=np.float32)[:, None, None]
        blended = (1 - w) * b1 + w * b2

        out = np.concatenate([a, blended, c], axis=0)

    return out


def build_training_pose_dir_with_selfstitch(
    base_pose_dir="./pose-data-aug",
    out_pose_dir="./pose-data-train",
    fade=15,
    target_frames=300,
    add_noise_std=0.0015,
    seed=42,
    overwrite=False,
):
    """
    Creates a new pose directory for training:
      - copies ALL .npy files from base_pose_dir into out_pose_dir
      - generates self-stitched samples (3->9, 4->8, 5->10) from the base poses
        and writes them into out_pose_dir as additional .npy files

    Result: out_pose_dir becomes the single directory you train on.
    """
    rng = np.random.default_rng(seed)

    base_pose_dir = Path(base_pose_dir)
    out_pose_dir = Path(out_pose_dir)
    out_pose_dir.mkdir(parents=True, exist_ok=True)

    # Optional: clear output dir first
    if overwrite:
        for p in out_pose_dir.glob("*.npy"):
            p.unlink()

    # 1) Copy all base poses into out_pose_dir
    base_files = sorted(base_pose_dir.glob("*.npy"))
    if not base_files:
        raise RuntimeError(f"No .npy files found in {base_pose_dir}")

    copied = 0
    for p in base_files:
        dst = out_pose_dir / p.name
        if not dst.exists():
            shutil.copy2(p, dst)
            copied += 1

    print(f"✅ Copied {copied} base pose files into {out_pose_dir}")

    # 2) Generate self-stitched poses into the SAME out_pose_dir
    # base reps -> (repeat times, target reps)
    plan = {3: (3, 9), 4: (2, 8), 5: (2, 10)}
    written = {8: 0, 9: 0, 10: 0}

    for p in base_files:
        base_reps = int(p.name.split("_")[0])
        if base_reps not in plan:
            continue

        times, target_reps = plan[base_reps]

        kp = np.load(p).astype(np.float32)  # (T,33,3)

        stitched = self_stitch_pose(kp, times=times, fade=fade)

        if add_noise_std and add_noise_std > 0:
            stitched += rng.normal(0, add_noise_std,
                                   size=stitched.shape).astype(np.float32)

        stitched = resample_pose_to_length(stitched, target_len=target_frames)

        # Name it with new target reps prefix so your dataset label becomes 8/9/10
        out_name = f"{target_reps}_" + p.stem + f"__selfx{times}.npy"
        dst = out_pose_dir / out_name

        # avoid overwriting if rerun
        if dst.exists():
            continue

        np.save(dst, stitched.astype(np.float32))
        written[target_reps] += 1

    print(f"✅ Added self-stitched poses into {out_pose_dir}")
    print(f"   +8 reps:  {written[8]}")
    print(f"   +9 reps:  {written[9]}")
    print(f"   +10 reps: {written[10]}")

    return str(out_pose_dir)

## Balancing through Copying

In [9]:
from pathlib import Path
import shutil
import random
from collections import defaultdict


def build_pose_dataset_balanced_by_copy(
    src_pose_dir="./pose-data-aug",
    dst_pose_dir="./pose-data-balanced",
    target_per_class=10,
    seed=42,
    copy_originals=True,
):
    """
    Balance pose dataset by COPYING existing pose files (no augmentation).

    - Groups .npy files by label (prefix number)
    - Copies originals into dst_pose_dir (optional)
    - If a class has < target_per_class files, duplicates them round-robin
    - If a class has >= target_per_class files, does nothing
    """
    rng = random.Random(seed)

    src_pose_dir = Path(src_pose_dir)
    dst_pose_dir = Path(dst_pose_dir)
    dst_pose_dir.mkdir(parents=True, exist_ok=True)

    # --- gather source poses by class ---
    poses_by_label = defaultdict(list)
    for p in sorted(src_pose_dir.glob("*.npy")):
        try:
            y = int(p.name.split("_")[0])
        except Exception:
            continue
        poses_by_label[y].append(p)

    if not poses_by_label:
        raise RuntimeError(f"No .npy files found in {src_pose_dir}")

    print("Source pose counts per class:")
    for y in sorted(poses_by_label):
        print(f"  {y}: {len(poses_by_label[y])}")

    # --- copy originals first ---
    if copy_originals:
        copied = 0
        for y, poses in poses_by_label.items():
            for p in poses:
                dst = dst_pose_dir / p.name
                if not dst.exists():
                    shutil.copy2(p, dst)
                    copied += 1
        print(f"\n✅ Copied {copied} original pose files into {dst_pose_dir}")

    # --- count what we have now in dst ---
    existing_by_label = defaultdict(list)
    for p in dst_pose_dir.glob("*.npy"):
        try:
            y = int(p.name.split("_")[0])
        except Exception:
            continue
        existing_by_label[y].append(p)

    print("\nCounts after copying originals:")
    for y in sorted(poses_by_label):
        print(f"  {y}: {len(existing_by_label[y])}")

    # --- duplicate until target_per_class ---
    for y in sorted(poses_by_label):
        have = len(existing_by_label[y])
        if have >= target_per_class:
            print(
                f"\nClass {y}: already has {have} ≥ {target_per_class}, skipping.")
            continue

        need = target_per_class - have
        src_poses = poses_by_label[y]

        print(f"\nClass {y}: need {need} more copies.")

        for i in range(need):
            src_p = src_poses[i % len(src_poses)]
            dup_id = rng.randint(0, 10**9)

            out_name = f"{src_p.stem}__copy{dup_id}.npy"
            out_path = dst_pose_dir / out_name

            while out_path.exists():
                dup_id = rng.randint(0, 10**9)
                out_name = f"{src_p.stem}__copy{dup_id}.npy"
                out_path = dst_pose_dir / out_name

            shutil.copy2(src_p, out_path)
            existing_by_label[y].append(out_path)

        print(f"  -> class {y} now has {len(existing_by_label[y])}")

    print(f"\n✅ Balanced pose dataset written to: {dst_pose_dir}")
    return str(dst_pose_dir)

# Train your Model

### TODO 5

Training time! Please include your training code below.

As per above, please feel free (and encouraged) to rip out all of the below code and replace with your (much better) code.

The below should just be used as an example to get you started.

In [10]:
import torch.optim as optim


def train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

    return total_loss / len(train_loader), correct / total


def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            total_loss += criterion(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

    return total_loss / len(test_loader), correct / total


def train_model(epochs=5, lr=1e-3, pose_dir="./pose-data"):
    """Train and return your model."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")

    # model = PoseTemporalCNN(num_classes=10, num_joints=33, dim=3).to(device)
    model = PoseGRUClassifier(
        num_classes=11, input_dim=99, hidden=256).to(device)

    print("Instantiated model.\n")
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.9, patience=10
    )
    criterion = nn.CrossEntropyLoss()

    train_loader, val_loader = get_dataloaders(
        pose_dir=pose_dir, batch_size=32, val_split=0.01)
    print("Got dataloaders.\n")

    print("Go time. Let the training commence.\n")

    print("Validation Accuracy is only a dummy value, since val_split=0.01 and model is trained on entire dataset for maximum performance")

    for epoch in range(1, epochs + 1):

        train_loss, train_acc = train_epoch(
            model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)

        scheduler.step(val_loss)

        print(f"Epoch {epoch}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, LR: {optimizer.param_groups[0]['lr']:.6f}")

    return model



def save_model(model, path="model.pt"):
    """Save the model weights to a file."""
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

## Build Augmented Dataset

In [11]:


expanded_dir = build_augmented_dataset(
    src_dir="./video-data",
    dst_dir="./video-data-aug",
    copies_per_video=2,
    crop_scale_range=(0.80, 1.00),
    flip_prob=0.5,
)



Found 77 original videos
Augmented: 1_dksksjfwijf.mp4
Augmented: 2_dfsaeklnvvalkej.mp4
Augmented: 2_difficult_2.mp4
Augmented: 2_difficult_sdafkljsalkfj.mp4
Augmented: 2_dkdjwkndkfw.mp4
Augmented: 2_dkdmkejkeimdh.mp4
Augmented: 2_dkjd823kjf.mp4
Augmented: 2_dsalkfjalwkenlke.mp4
Augmented: 2_kling_20251205_Text_to_Video_On_a_sandy_4976_0.mp4
Augmented: 2_kling_20251206_Text_to_Video_Generate_a_71_1.mp4
Augmented: 2_sadfasjldkfjaseifj.mp4
Augmented: 2_sdafkjaslkclaksdjkas.mp4
Augmented: 2_sdfkjsaleijflaskdjf.mp4
Augmented: 2_sdjfhafsldkjhjk.mp4
Augmented: 2_sdkjdsflkjfwa.mp4
Augmented: 2_sdlfjlewlkjkj.mp4
Augmented: 2_sdlkjsaelijfksdjf.mp4
Augmented: 3_asldkfjalwieaskdfaskdf.mp4
Augmented: 3_dkk873lkjlksajdf.mp4
Augmented: 3_dsjlaeijlksjdfie.mp4
Augmented: 3_dsksdfjbvsdkj.mp4
Augmented: 3_dslkaldskjflakjs.mp4
Augmented: 3_ewdfkjwaeoihjlkasdjf.mp4
Augmented: 3_kling_20251205_Text_to_Video_In_a_grass_4697_0.mp4
Augmented: 3_kling_20251205_Text_to_Video_On_a_playg_5028_0.mp4
Augmented: 3_kl

## Cache Pose Data

In [12]:
cache_all_poses(video_dir="./video-data-aug", pose_dir="./pose-data-aug",
                target_frames=300, stride=2, include_visibility=True)
print("Cached Posedata in ./pose-data-aug")

Caching 2_dkdjwkndkfw__aug1.avi


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Caching 1_dksksjfwijf.mp4
Caching 3_asldkfjalwieaskdfaskdf.mp4
Caching 7_sadkjfkljekj__aug1.avi
Caching 3_dkk873lkjlksajdf__aug0.avi
Caching 3_asldkfjalwieaskdfaskdf__aug0.avi
Caching 3_kling_20251209_Text_to_Video_Generate_a_491_1.mp4
Caching 3_dsjlaeijlksjdfie__aug0.avi
Caching 3_kling_20251205_Text_to_Video_On_a_playg_5064_0__aug1.avi
Caching 3_ewdfkjwaeoihjlkasdjf.mp4
Caching 3_sdlkfjaleknaksej.mp4
Caching 4_kling_20251207_Text_to_Video_Generate_a_521_1__aug0.avi
Caching 5_sdfkljweoijlkjdsflkjweaij.mp4
Caching 3_kling_20251209_Text_to_Video_Generate_a_403_1.mp4
Caching 3_sadlfkjawelnflksdjf.mp4
Caching 2_kling_20251206_Text_to_Video_Generate_a_71_1.mp4
Caching 2_dsalkfjalwkenlke.mp4
Caching 4_kling_20251209_Text_to_Video_Generate_a_263_1__aug0.avi
Caching 4_kling_20251209_Text_to_Video_Generate_a_561_1__aug0.avi
Caching 2_sadfasjldkfjaseifj.mp4
Caching 4_sadflkjasldkjfalseij__aug0.avi
Caching 2_difficult_2.mp4
Caching 3_kling_20251205_Text_to_Video_On_a_playg_5028_0__aug1.avi
Cachi

## Stitch Videos and Balance

In [13]:
# Build Training Pose Directory with Self-Stitched Samples
train_pose_dir = build_training_pose_dir_with_selfstitch(
    base_pose_dir="./pose-data-aug",
    out_pose_dir="./pose-data-aug-stitched",
    fade=15,
    target_frames=300,
    overwrite=True,   # optional; clears old outputs if rerun
)

# Build Balanced Pose Dataset by Copying
balanced_pose_dir = build_pose_dataset_balanced_by_copy(
    src_pose_dir=train_pose_dir,
    dst_pose_dir="./pose-data-final",
    target_per_class=10,
    seed=42,
)

✅ Copied 231 base pose files into pose-data-aug-stitched
✅ Added self-stitched poses into pose-data-aug-stitched
   +8 reps:  63
   +9 reps:  102
   +10 reps: 6
Source pose counts per class:
  1: 3
  2: 48
  3: 102
  4: 63
  5: 6
  6: 6
  7: 3
  8: 63
  9: 102
  10: 6

✅ Copied 402 original pose files into pose-data-final

Counts after copying originals:
  1: 3
  2: 48
  3: 102
  4: 63
  5: 6
  6: 6
  7: 3
  8: 63
  9: 102
  10: 6

Class 1: need 7 more copies.
  -> class 1 now has 10

Class 2: already has 48 ≥ 10, skipping.

Class 3: already has 102 ≥ 10, skipping.

Class 4: already has 63 ≥ 10, skipping.

Class 5: need 4 more copies.
  -> class 5 now has 10

Class 6: need 4 more copies.
  -> class 6 now has 10

Class 7: need 7 more copies.
  -> class 7 now has 10

Class 8: already has 63 ≥ 10, skipping.

Class 9: already has 102 ≥ 10, skipping.

Class 10: need 4 more copies.
  -> class 10 now has 10

✅ Balanced pose dataset written to: pose-data-final


## Train and Save Model (Val Accuracy is meaningless because we train final model on full dataset --> for actual validation accuracy tune val_split to 0.2 instead of 0.01)

In [14]:
# setting a manual seed for reproducibility
torch.manual_seed(0)
model = train_model(epochs=200,
                    pose_dir="./pose-data-aug")

save_model(model, "mv-final-assignment-gru.pt")

Using device: cuda

Instantiated model.

Train: 229 videos, Val: 2 videos

Got dataloaders.

Go time. Let the training commence.

Validation Accuracy is only a dummy value, since val_split=0.01 and model is trained on entire dataset for maximum performance
Epoch 1/200 | Train Loss: 1.8245, Train Acc: 0.3712 | Val Loss: 0.9112, Val Acc: 0.5000, LR: 0.001000
Epoch 2/200 | Train Loss: 1.4730, Train Acc: 0.3450 | Val Loss: 1.2812, Val Acc: 0.5000, LR: 0.001000
Epoch 3/200 | Train Loss: 1.4011, Train Acc: 0.4192 | Val Loss: 1.1293, Val Acc: 0.5000, LR: 0.001000
Epoch 4/200 | Train Loss: 1.4262, Train Acc: 0.4279 | Val Loss: 1.0310, Val Acc: 0.5000, LR: 0.001000
Epoch 5/200 | Train Loss: 1.3442, Train Acc: 0.4017 | Val Loss: 1.0332, Val Acc: 0.5000, LR: 0.001000
Epoch 6/200 | Train Loss: 1.3266, Train Acc: 0.4192 | Val Loss: 1.0194, Val Acc: 0.5000, LR: 0.001000
Epoch 7/200 | Train Loss: 1.3623, Train Acc: 0.4061 | Val Loss: 0.9447, Val Acc: 0.5000, LR: 0.001000
Epoch 8/200 | Train Loss: 1.3

## Upload to Huggingface (Uncomment this Code in case you want to reupload model)

In [16]:
# --------------- Upload to Hugging Face Hub --------------
#!pip install -q huggingface_hub
# from huggingface_hub import login
# login("MY_HF_API_TOKEN")  # <-- replace with your token or use environment variable

# hf_username = "jonny-vr"


# def upload_to_hf(
#     local_path="mv-final-assignment-gru.pt",
#     repo_id="jonny-vr/mv-final-assignment-gru-notebook",   # <-- change if you want
#     filename_in_repo="mv-final-assignment-gru.pt",
#     commit_message="Upload trained GRU pose model",
#     private=False
# ):
#     api = HfApi()

#     # Create repo if it doesn't exist
#     api.create_repo(repo_id=repo_id, repo_type="model",
#                     exist_ok=True, private=private)

#     # Upload the file
#     api.upload_file(
#         path_or_fileobj=local_path,
#         path_in_repo=filename_in_repo,
#         repo_id=repo_id,
#         repo_type="model",
#         commit_message=commit_message,
#     )
#     print(f"Uploaded {local_path} to https://huggingface.co/{repo_id}")


# upload_to_hf(
#     local_path="mv-final-assignment-gru.pt",
#     repo_id=f"{hf_username}/mv-final-assignment-gru",
# )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  mv-final-assignment-gru.pt  :  20%|##        |  557kB / 2.73MB            

Uploaded mv-final-assignment-gru.pt to https://huggingface.co/jonny-vr/mv-final-assignment-gru
